In [1]:
import os,sys,subprocess,time
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import numpy as np
import pandas as pd
pd.set_option("display.float_format","{:.2f}".format)
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
pio.templates["mod"] = go.layout.Template(layout=dict(font=dict(family="Fira Code",size=20)))
pio.templates.default = "plotly_dark+mod"
from zipfile import ZipFile
from glob import glob
from scipy import stats
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler,OrdinalEncoder,MinMaxScaler,RobustScaler,OneHotEncoder
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV,train_test_split,StratifiedShuffleSplit
from sklearn.metrics import roc_auc_score,confusion_matrix,accuracy_score,f1_score,precision_recall_curve
import miceforest as mf
import tensorflow as tf
from tensorflow import keras
tf.get_logger().setLevel('ERROR')
%matplotlib inline

In [2]:
peng_lter = pd.read_csv('penguins_lter.csv')

In [3]:
peng_lter.head(3)

,studyName,Sample Number,Species,Region,Island,Stage,Individual ID,Clutch Completion,Date Egg,Culmen Length (mm),Culmen Depth (mm),Flipper Length (mm),Body Mass (g),Sex,Delta 15 N (o/oo),Delta 13 C (o/oo),Comments
0,PAL0708,1,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N1A1,Yes,11/11/07,39.10,18.70,181.00,3750.00,MALE,NaN,NaN,Not enough blood for isotopes.
1,PAL0708,2,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N1A2,Yes,11/11/07,39.50,17.40,186.00,3800.00,FEMALE,8.95,-24.69,NaN
2,PAL0708,3,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N2A1,Yes,11/16/07,40.30,18.00,195.00,3250.00,FEMALE,8.37,-25.33,NaN


In [4]:
peng_lter.shape

(344, 17)

In [5]:
peng_lter.describe()

,Sample Number,Culmen Length (mm),Culmen Depth (mm),Flipper Length (mm),Body Mass (g),Delta 15 N (o/oo),Delta 13 C (o/oo)
count,344.00,342.00,342.00,342.00,342.00,330.00,331.00
mean,63.15,43.92,17.15,200.92,4201.75,8.73,-25.69
std,40.43,5.46,1.97,14.06,801.95,0.55,0.79
min,1.00,32.10,13.10,172.00,2700.00,7.63,-27.02
25%,29.00,39.23,15.60,190.00,3550.00,8.30,-26.32
50%,58.00,44.45,17.30,197.00,4050.00,8.65,-25.83
75%,95.25,48.50,18.70,213.00,4750.00,9.17,-25.06
max,152.00,59.60,21.50,231.00,6300.00,10.03,-23.79


In [6]:
peng_lter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   studyName            344 non-null    object 
 1   Sample Number        344 non-null    int64  
 2   Species              344 non-null    object 
 3   Region               344 non-null    object 
 4   Island               344 non-null    object 
 5   Stage                344 non-null    object 
 6   Individual ID        344 non-null    object 
 7   Clutch Completion    344 non-null    object 
 8   Date Egg             344 non-null    object 
 9   Culmen Length (mm)   342 non-null    float64
 10  Culmen Depth (mm)    342 non-null    float64
 11  Flipper Length (mm)  342 non-null    float64
 12  Body Mass (g)        342 non-null    float64
 13  Sex                  334 non-null    object 
 14  Delta 15 N (o/oo)    330 non-null    float64
 15  Delta 13 C (o/oo)    331 non-null    flo

As comments column is heavily downsized and missing we can omit it<br>
As this is heavily specie-related dataset lets clean the species column first<br>
since there is only one Region we can drop it<br>
since there is only one Stage we can drop it

In [7]:
def preprocess(data:pd.DataFrame):
   df = data.drop(["Comments"],axis=1)
   df["Species"] = df.Species.str.split(n=1,expand=True)[0]
   df[['N','A']] =df['Individual ID'].str.split(r'(\d+)',expand=True)[[1,3]]
   df['Date Egg'] = pd.to_datetime(df['Date Egg'],format='mixed')
   df.drop(columns=['Region','Stage','Individual ID','Sample Number'],inplace=True)
   original_columns =df.columns.to_list()
   df.columns = ['study','species','island','clutch','date','culmenL','culmenD','flipperL','bmass','sex','delta_15n','delta_13c','N','A']
   df[['N','A']] =df[['N','A']].astype(np.float32)
   return df

<span style="font-family:Fira Code">
<font size=4>

|Column Name|Description|Type|
|:----------|:----------|:---|
|StudyName|Sampling expedition from which data were collected, generated, etc.|categorical|
|Sample Number|an integer denoting the continuous numbering sequence for each sample|categorical|
|Species|a character string denoting the penguin species|categorical|
|Region|a character string denoting the region of Palmer LTER sampling grid|categorical|
|Island|a character string denoting the island near Palmer Station where samples were collected|categorical|
|Stage|a character string denoting reproductive stage at sampling|categorical|
|IndividualID|a character string denoting the unique ID for each individual in dataset [N,A]|categorical|
|Clutch Completion|a character string denoting if the study nest observed with a full clutch, i.e., 2 eggs|categorical|
|Date Egg|a date denoting the date study nest observed with 1 egg (sampled)|continuous|
|Culmen Length|a number denoting the length of the dorsal ridge of a bird's bill (millimeters)|continuous|
|Culmen Depth|a number denoting the depth of the dorsal ridge of a bird's bill (millimeters)|continuous|
|Flipper Length|an integer denoting the length penguin flipper (millimeters)|continuous|
|Body Mass|an integer denoting the penguin body mass (grams)|continuous|
|Sex|a character string denoting the sex of an animal|categorical|
|Delta 15 N|a number denoting the measure of the ratio of stable isotopes 15N:14N|continuous|
|Delta 13 C|a number denoting the measure of the ratio of stable isotopes 13C:12C|continuous|

In [8]:
temp_df = preprocess(peng_lter)

In [9]:
temp_df.head()

,study,species,island,clutch,date,culmenL,culmenD,flipperL,bmass,sex,delta_15n,delta_13c,N,A
0,PAL0708,Adelie,Torgersen,Yes,2007-11-11,39.10,18.70,181.00,3750.00,MALE,NaN,NaN,1.00,1.00
1,PAL0708,Adelie,Torgersen,Yes,2007-11-11,39.50,17.40,186.00,3800.00,FEMALE,8.95,-24.69,1.00,2.00
2,PAL0708,Adelie,Torgersen,Yes,2007-11-16,40.30,18.00,195.00,3250.00,FEMALE,8.37,-25.33,2.00,1.00
3,PAL0708,Adelie,Torgersen,Yes,2007-11-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.00,2.00
4,PAL0708,Adelie,Torgersen,Yes,2007-11-16,36.70,19.30,193.00,3450.00,FEMALE,8.77,-25.32,3.00,1.00


In [10]:
temp_df.loc[336,'sex'] = pd.NA
temp_na = temp_df.dropna()

<span style="font-family:Fira Code">
<h1>
Univariate Analysis

<span style="font-family:Fira Code">
<h2>
Continuous

<span style="font-family:Fira Code">
<h3>
Date

In [11]:
print("Study: ",temp_na.study.unique()[0],"and is carried on",temp_na.loc[temp_na.study == temp_na.study.unique()[0],'date'].dt.year.unique()[0])
print("Study: ",temp_na.study.unique()[1],"and is carried on",temp_na.loc[temp_na.study == temp_na.study.unique()[1],'date'].dt.year.unique()[0])
print("Study: ",temp_na.study.unique()[2],"and is carried on",temp_na.loc[temp_na.study == temp_na.study.unique()[2],'date'].dt.year.unique()[0])

Study:  PAL0708 and is carried on 2007
Study:  PAL0809 and is carried on 2008
Study:  PAL0910 and is carried on 2009


<span style="font-family:Fira Code">
There are three years [ 2007  2008  2009 ] and three studies which are carried on different times

In [12]:
years = temp_na.date.dt.year.unique()
studies = temp_na.study.unique()
colors_list = px.colors.qualitative.Plotly[:3]
fig = go.Figure()
for y,col,i in zip(years,colors_list,range(3)):
    df = temp_na.loc[temp_na['date'].dt.year == y]
    fig.add_trace(go.Scatter(x=df.date,y=[1]*df.shape[0],mode="lines",name=f"{y}",line=dict(color=col)))
    fig.add_trace(go.Scatter(x=np.r_[df.date.min(),df.date.max()],y=[1]*2,mode="markers",name="i",marker=dict(color=col)))
    fig.add_annotation(text=f"{studies[i]}",x=df.date.mean(),y=0.65,yref="paper",showarrow=False,font=dict(color=colors_list[i]),bordercolor=colors_list[i],borderpad=5)
for trace in fig.data:
    if trace['name'] == 'i':
        trace['showlegend'] = False
fig.update_layout(title="Date Range")
fig.update_yaxes(showticklabels=False)
fig.show()

<span style="font-family:Fira Code">
<h3>
Physical Attributes

In [13]:
phy_cols = ['culmenL','culmenD','flipperL','bmass']
temp_na[phy_cols].describe()

,culmenL,culmenD,flipperL,bmass
count,324.00,324.00,324.00,324.00
mean,44.05,17.13,201.22,4213.97
std,5.48,1.97,13.96,809.28
min,32.10,13.10,172.00,2700.00
25%,39.50,15.57,190.00,3550.00
50%,44.95,17.30,197.00,4050.00
75%,48.70,18.60,213.00,4800.00
max,59.60,21.50,231.00,6300.00


In [14]:
def plot_qq(_):
    d = stats.zscore(temp_na[_].to_numpy())
    data = sm.qqplot(d,line='45',fit=True,dist=stats.norm).gca().lines
    plt.close()
    return data

def shapiro_test(_):
    statistic,p_value = stats.shapiro(temp_na[_].to_numpy())
    if p_value > 0.05:
        return p_value,"Shapiro Normality test H0 fail to reject"
    else:
        return p_value,"Shapiro Normality test H0 reject"

In [15]:
fig = make_subplots(rows=4,cols=3,subplot_titles=("Culmen Length(mm)","Culmen Length qq-plot","Shapiro test Culmen Length","Culmen Depth(mm)","Culmen Depth qq-plot","Shapiro test Culmen Depth","Flipper Length(mm)","Flipper Length qq-plot","Shapiro test Flipper Length","Body Mass(g)","Body Mass qq-plot","Shapiro test BodyMass"),vertical_spacing=0.05)
colors_list = px.colors.qualitative.Plotly[:4]
for i,color_ in enumerate(colors_list):
    d = plot_qq(phy_cols[i])
    p_val,text_ = shapiro_test(phy_cols[i])
    fig.add_trace(go.Histogram(x=temp_na[phy_cols[i]],marker=dict(color=color_)),row=i+1,col=1)
    fig.append_trace(go.Scatter(x=d[0].get_xdata(),y=d[0].get_ydata(),mode="markers",showlegend=False,marker=dict(color=color_)),row=i+1,col=2)
    fig.append_trace(go.Scatter(x=d[1].get_xdata(),y=d[1].get_ydata(),mode="lines",showlegend=False,line=dict(color="white")),row=i+1,col=2)
    fig.add_annotation(text=f"p_value of shapiro: {p_val}<br>{text_}",row=i+1,col=3)
    fig.update_xaxes(showgrid=False,showticklabels=False,row=i+1,col=3)
    fig.update_yaxes(showgrid=False,showticklabels=False,row=i+1,col=3)
fig.update_layout(showlegend=False,height=1500,title=dict(text="Physical Attributes Histogram and Q-Q plot",font=dict(size=30)),margin=dict(t=150))
fig.show()

<span style="font-family:Fira Code">
<h3>
Experiment Outcomes

In [16]:
exp_cols = ['delta_15n','delta_13c']
temp_na[exp_cols].describe()

,delta_15n,delta_13c
count,324.00,324.00
mean,8.74,-25.69
std,0.55,0.79
min,7.63,-27.02
25%,8.30,-26.33
50%,8.66,-25.84
75%,9.18,-25.06
max,10.03,-23.89


<span style="font-family:Fira Code">

$\delta^{15}N$ and $\delta^{13}C$ are the ratio of the the two stable isotopes of the respective elements with respective to atmosphere
- If the number is positive that means relative enrichment to the atmosphere
- If the number is negative that means relative depletion to the atmosphere
- Typical insignificant range is from [-20,80]

In [17]:
fig = make_subplots(rows=2,cols=3,subplot_titles=("Delta 15N Ratio Histogram","Delta 15N qq-plot","Shapiro test Delta 15N","Delta 13C Histogram","Delta 13C qq-plot","Shapiro test Delta 13C"))
colors_list = px.colors.qualitative.Plotly[:2]
for i,color_ in enumerate(colors_list):
    d = plot_qq(exp_cols[i])
    p_val,text_ = shapiro_test(exp_cols[i])
    fig.add_trace(go.Histogram(x=temp_na[exp_cols[i]],marker=dict(color=color_)),row=i+1,col=1)
    fig.append_trace(go.Scatter(x=d[0].get_xdata(),y=d[0].get_ydata(),mode="markers",showlegend=False,marker=dict(color=color_)),row=i+1,col=2)
    fig.append_trace(go.Scatter(x=d[1].get_xdata(),y=d[1].get_ydata(),mode="lines",showlegend=False,line=dict(color="white")),row=i+1,col=2)
    fig.add_annotation(text=f"p_value of shapiro: {p_val}<br>{text_}",row=i+1,col=3)
    fig.update_xaxes(showgrid=False,showticklabels=False,row=i+1,col=3)
    fig.update_yaxes(showgrid=False,showticklabels=False,row=i+1,col=3)
fig.update_layout(showlegend=False,height=1000,title=dict(text="Obtained Data Histogram and Q-Q plot",font=dict(size=30)),margin=dict(t=150))
fig.show()

<span style="font-family:Fira Code">
<h2>
Categorical

In [18]:
cat_cols = ['study','species','island','clutch','sex']
temp_na[cat_cols].describe()

,study,species,island,clutch,sex
count,324,324,324,324,324
unique,3,3,3,2,2
top,PAL0910,Adelie,Biscoe,Yes,FEMALE
freq,116,139,162,290,163


In [19]:
colors_list = px.colors.qualitative.Plotly
fig = make_subplots(cols=2,specs=[[{},{'type':'domain'}]])
fig.add_trace(go.Pie(labels=temp_na[cat_cols[0]],visible=True,textinfo="percent+label"),row=1,col=2)
fig.add_trace(go.Bar(x=temp_na[cat_cols[0]].value_counts().index,y=temp_na[cat_cols[0]].value_counts(),visible=True,texttemplate="%{y}",textposition="outside",marker=dict(color=colors_list[:temp_na[cat_cols[0]].nunique()])),row=1,col=1)
for col in cat_cols[1:]:
    fig.add_trace(go.Pie(labels=temp_na[col],visible=False,textinfo="percent+label"),row=1,col=2)
    fig.add_trace(go.Bar(y=temp_na[col].value_counts(),x=temp_na[col].value_counts().index,visible=False,texttemplate="%{y}",textposition="outside",marker=dict(color=colors_list[:temp_na[col].nunique()])),row=1,col=1)

col_names_dict=dict(study="Study",species="Species",island="Island",clutch="Clutch",sex="Sex")
buttons = []
for i,col in enumerate(cat_cols):
    visible = [False]*10
    visible[i*2] = True
    visible[i*2+1] = True
    d = dict(method="restyle",label=col_names_dict[col],visible=True,args=[{"visible":visible}])
    buttons.append(d)

buttons
updatemenus = [dict(
    type="buttons",
    direction="left",
    buttons=buttons,
    showactive=True,
    x = 0,
    y = 1.02,
    xanchor="left",
    yanchor="bottom",
    font=dict(size=40,color="red")
)]
fig.update_layout(updatemenus=updatemenus,height=1000,showlegend=False)
fig.show()

<font face="Fira Code">
<h1>
Bi-Variate Analysis

<font size = 5 face = "Fira Code">
Main columns of interest for continuous are Isotopes and physical attributes<br>
Main columns of interest for categorical are species,sex,island

In [75]:
corr_threshold = 0.6
corr_img = pd.get_dummies(temp_na.drop(columns=['date']),prefix="",prefix_sep="",dtype=np.float64).corr(method="spearman").copy()
corr_img_all = corr_img.copy()
cond = (corr_img > corr_threshold) | (corr_img < -corr_threshold)
corr_img = corr_img.where(cond,0)
np.fill_diagonal(corr_img.to_numpy(),0)
fig = px.imshow(corr_img,height=1000,title=f"Correlation showing only the highly correlated elements greater than {corr_threshold} and less than -{corr_threshold}",color_continuous_scale="GnBu")
fig.add_hline(y=5.5,line=dict(dash="dash",color="rgba(255,255,255,0.5)"))
fig.add_vline(x=5.5,line=dict(dash="dash",color="rgba(255,255,255,0.5)"))
fig.add_annotation(text="categorical quadrant",x=1.35,y=0.3,showarrow=False,font=dict(color="white"),xref="x domain",yref="y domain")
fig.add_annotation(text="continuous quadrant",x=1.34,y=0.9,showarrow=False,font=dict(color="white"),xref="x domain",yref="y domain")
fig.show()

<font face="Fira Code" size=5>
There are quite a number of high correlations, but with adjusting the correlation threshold we can see the most important ones.<br>

- ~~Culmen Lenght v/s Species~~
- Culmen Depth v/s Species
- Flipper Length v/s Body Mass
- Flipper Lenght v/s Species
- Sex v/s A
- $\delta^{15}N$ v/s Species
- $\delta^{13}C$ v/s Species
- Island v/s Species
- $\delta^{15}N$ v/s Island
- Island v/s Biomass
- Island v/s Culmen Depth


In [87]:
px.histogram(temp_na,x="species",y="culmenL",histfunc="avg",color="species",height=700,text_auto=True).update_traces(textposition="outside",texttemplate="Avg : %{y}")

In [89]:
px.histogram(temp_na,x="species",y="culmenD",color='species',histfunc="avg",text_auto=True,height=700).update_traces(textposition="outside",texttemplate="Avg : %{y}")

In [21]:
# ord_enc = OrdinalEncoder().set_output(transform='pandas')
# temp_df[['study','species','island','clutch','sex']] = ord_enc.fit_transform(temp_df[['study','species','island','clutch','sex']])
# temp_df.head()
# kds = mf.ImputationKernel(data=temp_df.drop(columns=['date']),datasets=5,random_state=1991)
# kds.mice(1)
# plt.rcParams["figure.figsize"] = (20,10)
# kds.plot_imputed_distributions(datasets=1)
# temp_df = kds.complete_data(dataset=1)
# GradientBoostingClassifier().get_params()
# gird_params = dict(
#     learnig_rate=np.logspace(-6,-1,6),
#     max_depth=np.arange(3,16),
#     max_leaf_nodes=np.arange(8,32),
#     n_estimators=np.arange(100,500,50)
#     )